# Homework 2 - Kaggle

2018180042 문승벽

이번 과제에서는 데이터 사이언스 학습 사이트인 Kaggle의 Titanic 사건을 주제로 한 예제를 풀어보도록 하겠다.
Kaggle의 Titanic 예제에서는 기본적으로 타이타닉 사고 당시의 승객 명단 데이터를 담은 csv 파일이 제공되는데, 생존자의 이름, 성별, 나이, 티켓요금, 생사여부 등의 정보가 포함되어 있다.
이 예제는 이 csv에 포함된 데이터를 통해 '''생존자의 생사여부'''와 '''다른 데이터들 간의 연관성'''을 분석하여 생존에 영향을 미치는 요소를 찾아내는 것을 목표로 하고 있다.

csv 시트의 각 컬럼은 다음과 같다.

* Survived: 생존 여부 => 0 = No, 1 = Yes
* pclass: 티켓 등급 => 1 = 1st, 2 = 2nd, 3 = 3rd
* Sex: 성별
* Age: 나이
* Sibsp: 함께 탑승한 형제자매, 배우자의 수
* Parch: 함께 탑승한 부모, 자식의 수
* Ticket: 티켓 번호
* Fare: 운임
* Cabin: 객실 번호
* Embarked: 탑승 항구 => C = Cherbourg, Q = Queenstown, S = Southampton

원 링크 : https://www.kaggle.com/competitions/titanic/data
문제 해석 : https://developers.ascentnet.co.jp/2017/11/24/kaggle-process-review/



# 1. 코드 분석 & 주석

In [ ]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split

# Titanic 데이터의 커스텀 데이터셋 정의
class TitanicDataset(Dataset):
  def __init__(self, X, y):
    self.X = torch.FloatTensor(X) # float
    self.y = torch.LongTensor(y) # long int

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    feature = self.X[idx]
    target = self.y[idx]
    return {'input': feature, 'target': target} 

  def __str__(self):
    str = "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
      len(self.X), self.X.shape, self.y.shape
    )
    return str
  

학습 데이터를 처리하기 위한 커스텀 데이터셋 클래스를 정의한 부분이다. 이 클래스는 Kaggle로부터 주어진 데이터를 PyTorch의 데이터 로더에서 사용할 수 있는 형태로 변환하는 부분이다.

In [ ]:

# 테스트 데이터를 위한 커스텀 데이터셋
class TitanicTestDataset(Dataset):
  def __init__(self, X):
    self.X = torch.FloatTensor(X) # float

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    feature = self.X[idx]
    return {'input': feature}

  def __str__(self):
    str = "Data Size: {0}, Input Shape: {1}".format(
      len(self.X), self.X.shape
    )
    return str


바로 위의 코드 블록과 비슷한 내용이다. 해당 코드 부분은 테스트 데이터를 위한 커스텀 데이터셋 클래스를 정의한 부분이다. 

In [ ]:

# Titanic 데이터 전처리 함수 정의
def get_preprocessed_dataset():
    CURRENT_FILE_PATH = os.path.dirname(os.path.abspath(__file__)) # 현재 파일의 경로를 가져오기

    # 학습 데이터와 테스트 데이터 파일 경로 설정
    train_data_path = os.path.join(CURRENT_FILE_PATH, "train.csv")
    test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv")

    train_df = pd.read_csv(train_data_path) # CSV 파일에서 학습 데이터, 테스트 데이터 읽어오기
    test_df = pd.read_csv(test_data_path)

    all_df = pd.concat([train_df, test_df], sort=False) # 학습 데이터와 테스트 데이터를 병합.


    # 데이터 전처리 함수들을 순차적으로 적용
    all_df = get_preprocessed_dataset_1(all_df)

    all_df = get_preprocessed_dataset_2(all_df)

    all_df = get_preprocessed_dataset_3(all_df)

    all_df = get_preprocessed_dataset_4(all_df)

    all_df = get_preprocessed_dataset_5(all_df)

    all_df = get_preprocessed_dataset_6(all_df)

    train_X = all_df[~all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True) # 학습 데이터와 테스트 데이터로 다시 분리
    train_y = train_df["Survived"]

    test_X = all_df[all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)

    dataset = TitanicDataset(train_X.values, train_y.values) # 정의한 클래스를 바탕으로 커스텀 데이터셋 생성
    #print(dataset)
    train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2]) # 학습 데이터를 학습과 검증 데이터셋으로 분리
    test_dataset = TitanicTestDataset(test_X.values) # 정의한 클래스를 바탕으로 테스트 데이터셋 생성
    #print(test_dataset)

    return train_dataset, validation_dataset, test_dataset

이 부분은 Titanic 데이터를 전처리하고 학습, 검증, 테스트 데이터셋을 생성하는데 사용된다.
경로에서 학습 데이터 및 테스트 데이터의 csv파일을 가져와서 필요한 내용을 읽어들이고 병합한 후 전처리를 실시한다. 전처리가 끝나면 병합한 데이터 중 생사여부가 정의된 데이터만 다시 학습 데이터와 테스트 데이터로 분리하고, 학습 데이터용 및 테스트 데이터용 커스텀 데이터셋을 생성한다. 키 "Survived"를 이용하면 생사여부가 정의된 데이터만 가져올 수 있다.

학습 데이터는 다시 학습용과 검증용 데이터로 분리시킨다.

In [ ]:
# 전처리 프로세스

def get_preprocessed_dataset_1(all_df): 
    # Pclass별 Fare 평균값을 사용하여 Fare 결측치 메우기
    # Pclass에 따른 Fare의 평균값을 계산하고 누락된 Fare 값을 해당 Pclass의 평균값으로 대체.
    Fare_mean = all_df[["Pclass", "Fare"]].groupby("Pclass").mean().reset_index()
    Fare_mean.columns = ["Pclass", "Fare_mean"]
    all_df = pd.merge(all_df, Fare_mean, on="Pclass", how="left")
    all_df.loc[(all_df["Fare"].isnull()), "Fare"] = all_df["Fare_mean"]

    return all_df


def get_preprocessed_dataset_2(all_df):
    # name을 세 개의 컬럼으로 분리하여 다시 all_df에 합침
    # Name 컬럼을 콜론과 마침표를 기준으로 세 부분으로 나누고, 이를 다시 all_df에 추가.
    name_df = all_df["Name"].str.split("[,.]", n=2, expand=True)
    name_df.columns = ["family_name", "honorific", "name"]
    name_df["family_name"] = name_df["family_name"].str.strip()
    name_df["honorific"] = name_df["honorific"].str.strip()
    name_df["name"] = name_df["name"].str.strip()
    all_df = pd.concat([all_df, name_df], axis=1)

    return all_df


def get_preprocessed_dataset_3(all_df):
    # honorific별 Age 평균값을 사용하여 Age 결측치 메우기
    # Honorific에 따른 Age의 중앙값을 계산하고, 누락된 Age값을 해당 호칭의 중앙값으로 대체.
    honorific_age_mean = all_df[["honorific", "Age"]].groupby("honorific").median().round().reset_index()
    honorific_age_mean.columns = ["honorific", "honorific_age_mean", ]
    all_df = pd.merge(all_df, honorific_age_mean, on="honorific", how="left")
    all_df.loc[(all_df["Age"].isnull()), "Age"] = all_df["honorific_age_mean"]
    all_df = all_df.drop(["honorific_age_mean"], axis=1)

    return all_df


def get_preprocessed_dataset_4(all_df):
    # 가족수(family_num) 컬럼 새롭게 추가
    all_df["family_num"] = all_df["Parch"] + all_df["SibSp"]

    # 혼자탑승(alone) 컬럼 새롭게 추가
    # 가족이 없는 승객은 alone 컬럼을 1로 표시하고, 누락된 값을 0으로 대체.
    all_df.loc[all_df["family_num"] == 0, "alone"] = 1
    all_df["alone"].fillna(0, inplace=True)

    # 학습에 불필요한 컬럼 제거
    all_df = all_df.drop(["PassengerId", "Name", "family_name", "name", "Ticket", "Cabin"], axis=1)

    return all_df


def get_preprocessed_dataset_5(all_df):
    # honorific 값 개수 줄이기
    # honorific 값이 "Mr", "Miss", "Mrs", "Master"가 아닌 경우 "other"로 변경. 
    # Embarked 컬럼의 누락된 값을 "missing"으로 대체.
    all_df.loc[
    ~(
            (all_df["honorific"] == "Mr") |
            (all_df["honorific"] == "Miss") |
            (all_df["honorific"] == "Mrs") |
            (all_df["honorific"] == "Master")
    ),
    "honorific"
    ] = "other"
    all_df["Embarked"].fillna("missing", inplace=True)

    return all_df


def get_preprocessed_dataset_6(all_df):
    # 카테고리 변수를 LabelEncoder를 사용하여 수치값으로 변경하기
    # 데이터 프레임의 object 타입 카테고리 변수들을 LabelEncoder를 사용하여 수치값으로 변환.
    category_features = all_df.columns[all_df.dtypes == "object"]
    from sklearn.preprocessing import LabelEncoder
    for category_feature in category_features:
        le = LabelEncoder()
        if all_df[category_feature].dtypes == "object":
          le = le.fit(all_df[category_feature])
          all_df[category_feature] = le.transform(all_df[category_feature])

    return all_df


이 부분은 Titanic 데이터셋의 전처리 과정을 6단계에 걸쳐 수행한다. 각 명칭 끝의 숫자가 순서를 의미한다.

1번째 단계에서는 Pclass (티켓 클래스) 별 Fare (티켓 요금) 평균값을 사용하여 Fare 결축치를 메운다. Pclass에 따른 Fare의 평균값을 계산하고 누락된 Fare값을 해당 Pclass의 평균값으로 대체한다.
2번째 단계에서는 name을 콜론과 마침표를 기준으로 세 개의 컬럼으로 분리하여 다시 all_df에 합친다. 
3번째 단계에서는 honorific (호칭) 별 age 평균값을 이용하여 age의 결측치를 메운다. honorific에 따른 age의 중앙값을 계산하고, 누락된 age값을 해당 호칭의 중앙값으로 대체한다.
4번째 단계에서는 가족수를 나타내는 family_num 컬럼, 혼자 탑승했는지 여부를 나타내는 alone 컬럼을 새롭게 추가하고, 모델 학습에 불필요한 PassengerId, Name, family_name, name, Ticket, Cabin 컬럼을 제거한다.
5번째 단계에서는 honorific의 개수를 줄인다. honorific 값이 Mr, Miss, Mrs, Master가 아닌 경우 other로 변경한다. Embarked(정박지) 컬럼의 누락된 겂은 missing으로 대체한다.
마지막 단계에서는 데이터프레임의 object 타입의 카테고리 변수들을 LabelEncoder를 사용하여 수치값으로 변환한다.

이러한 전처리 단계는 데이터의 품질을 향상시키고 모델 학습을 위해 데이터를 더욱 적합하게 만들어 준다.

In [ ]:

from torch import nn
class MyModel(nn.Module): # 사용자 정의 모델
  def __init__(self, n_input, n_output):
    super().__init__()

    # 모델 정의: 입력 크기를 n_input, 출력 크기를 n_output으로 설정
    self.model = nn.Sequential( 
      nn.Linear(n_input, 30), # 입력 레이어 : 입력 특성을 30개의 뉴런으로 변환
      nn.ReLU(), # ReLU 활성화
      nn.Linear(30, 30), # 은닉 레이어 : 30개의 뉴런을 가진 은닉 레이어
      nn.ReLU(), # ReLU 활성화
      nn.Linear(30, n_output), # 출력 레이어: n_output 크기의 출력을 생성
    )

  def forward(self, x):
    x = self.model(x) # 모델의 순전파 (forward) 연산 정의
    return x

# 모델 테스트 함수 정의
def test(test_data_loader):
  print("[TEST]")
  batch = next(iter(test_data_loader)) # 테스트 데이터로더에서 배치 가져오기
  print("{0}".format(batch['input'].shape))
  my_model = MyModel(n_input=11, n_output=2) # 정의한 MyModel 클래스로 모델 인스턴스 생성
  output_batch = my_model(batch['input']) # 모델을 사용하여 테스트 데이터의 예측 생성 
  prediction_batch = torch.argmax(output_batch, dim=1) # 예측값 중 가장 높은 확률을 가진 클래스 선택
  for idx, prediction in enumerate(prediction_batch, start=892): # 예측 결과 출력
      print(idx, prediction.item())


이 부분에서는 PyTorch를 사용하여 신경망 모델을 정의하고, 모델을 사용하여 테스트 데이터의 예측을 수행하는 역할을 한다. MyModel은 입력 크기와 출력 크기를 인수로 받아 다층 퍼셉트론 신경망 모델을 생성한다. 모델이 정의되면 test 함수를 사용하여 모델을 테스트하고 예측 결과를 출력한다.

In [ ]:

if __name__ == "__main__":
  train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset() # 데이터 전처리 수행 결과 가져오기

  # 데이터셋 크기 및 구성 출력.
  print("train_dataset: {0}, validation_dataset.shape: {1}, test_dataset: {2}".format(
    len(train_dataset), len(validation_dataset), len(test_dataset)
  ))
  print("#" * 50, 1)

  # 훈련 데이터셋의 샘플 출력
  for idx, sample in enumerate(train_dataset):
    print("{0} - {1}: {2}".format(idx, sample['input'], sample['target']))

  print("#" * 50, 2)

  # DataLoader를 사용하여 미니배치로 데이터 로드.
  train_data_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
  validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=16, shuffle=True)
  test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

이 부분에서는 데이터 전처리 함수를 사용하여 훈련, 검증, 그리고 테스트 데이터셋을 가져온다. 가져온 데이터셋의 크기 및 구성을 출력하여 확인시키고, 훈련 데이터셋의 일부 샘플을 출력하여 데이터 형태를 확인한다. DataLoader를 사용하여 데이터셋을 미니배치로 로드한다. 

In [ ]:

  # 훈련 데이터셋을 사용하여 모델 학습 과정을 시작
  print("[TRAIN]")
  for idx, batch in enumerate(train_data_loader):
    print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape)) # 각 미니배치의 크기와 목표값 크기 출력

  # 검증 데이터셋을 사용하여 모델의 성능을 평가하는 단계 시작
  print("[VALIDATION]")
  for idx, batch in enumerate(validation_data_loader):
    print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape)) # 각 미니배치의 크기와 목표값 크기 출력

  print("#" * 50, 3) 

  test(test_data_loader)
  # test 함수를 호출하여 테스트 데이터로 모델 평가 후 결과 출력

이 부분은 Train 과 Validation, 즉 훈련과 검증의 두 부분으로 나뉘어져 있다. 훈련 부분에서는 모델을 학습하기 위해 훈련 데이터셋을 미니배치로 나누고, 각 미니배치의 입력 데이터와 목표값 데이터의 크기를 출력한다.
검증 부분에서는 검증 데이터셋을 사용하여 모델의 성능을 평가하는 단계를 시작, 마찬가지로 미니배치의 크기와 목표값 크기를 출력한다. 최종적으로 test 함수를 호출하여 테스트 데이터로 모델 평가 후 결과를 출력한다.




# 2. titanic 딥러닝 모델 훈련 코드 및 Activation Function 변경해보기

In [7]:
import torch
from torch import nn, optim
from torch.utils.data import random_split, DataLoader
from datetime import datetime
import wandb
import argparse
from pathlib import Path
import sys

# 기본 경로 설정
BASE_PATH = str(Path(__file__).resolve().parent.parent.parent)
sys.path.append(BASE_PATH)
import titanic_dataset as td # 실제 모듈 이름으로 변경

def get_data():
    # 데이터 전처리 모듈의 함수를 호출하여 데이터 가져오기
    train_dataset, validation_dataset, test_dataset = td.get_preprocessed_dataset()

    train_data_loader = DataLoader(dataset=train_dataset, batch_size=wandb.config.batch_size, shuffle=True)
    validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=wandb.config.batch_size, shuffle=True)
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

    return train_data_loader, validation_data_loader, test_data_loader

# 모델 클래스 정의
class MyModel(nn.Module):
    def __init__(self, n_input, n_output):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]),
            nn.ReLU(),
            nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]),
            nn.ReLU(),
            nn.Linear(wandb.config.n_hidden_unit_list[1], n_output),
        )

    def forward(self, x):
        x = self.model(x)
        return x

# 훈련 루프 함수 수정
def training_loop(model, optimizer, train_data_loader, validation_data_loader):
    n_epochs = wandb.config.epochs
    loss_fn = nn.CrossEntropyLoss()  # 분류 작업을 위한 손실 함수로 수정
    next_print_epoch = 100

    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        num_trains = 0
        for train_batch in train_data_loader:
            output_train = model(train_batch['input'])
            loss = loss_fn(output_train, train_batch['target'])
            loss_train += loss.item()
            num_trains += 1

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        loss_validation = 0.0
        num_validations = 0
        with torch.no_grad():
            for validation_batch in validation_data_loader:
                output_validation = model(validation_batch['input'])
                loss = loss_fn(output_validation, validation_batch['target'])
                loss_validation += loss.item()
                num_validations += 1

        wandb.log({
            "Epoch": epoch,
            "Training loss": loss_train / num_trains,
            "Validation loss": loss_validation / num_validations
        })

        if epoch >= next_print_epoch:
            print(
                f"Epoch {epoch}, "
                f"Training loss {loss_train / num_trains:.4f}, "
                f"Validation loss {loss_validation / num_validations:.4f}"
            )
            next_print_epoch += 100

# 모델과 옵티마이저 생성 함수
def get_model_and_optimizer():
    my_model = MyModel(n_input=11, n_output=2)
    optimizer = optim.SGD(my_model.parameters(), lr=wandb.config.learning_rate)

    return my_model, optimizer

def main(args):
    current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

    config = {
        'epochs': args.epochs,
        'batch_size': args.batch_size,
        'learning_rate': 1e-3,  # 원하는 학습률로 변경
        'n_hidden_unit_list': [30, 30],
    }

    wandb.login()

    wandb.init(
        mode="online" if args.wandb else "disabled",
        project="dl-titanic",
        notes="My first wandb experiment",
        tags=["my_model", "titanic", "kaggle"],
        name=current_time_str,
        config=config
    )


    train_data_loader, validation_data_loader, test_data_loader = get_data()
    linear_model, optimizer = get_model_and_optimizer()

    wandb.watch(linear_model)

    training_loop(
        model=linear_model,
        optimizer=optimizer,
        train_data_loader=train_data_loader,
        validation_data_loader=validation_data_loader,
    )
    wandb.finish()

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--wandb", action=argparse.BooleanOptionalAction, default=True, help="True or False"
    )
    parser.add_argument(
        "-b", "--batch_size", type=int, default=16, help="Batch size (int, default: 16)"  # 원하는 배치 크기로 수정
    )
    parser.add_argument(
        "-e", "--epochs", type=int, default=1_000, help="Number of training epochs (int, default: 1,000)"  # 원하는 에폭으로 수정
    )
    args = parser.parse_args()
    main(args)


NameError: name '__file__' is not defined

f_my_model_training_with_argparse_wandb.py를 이용하여 타이타닉 데이터셋에 맞게 내용을 수정해 보았다. 해당 코드는 py 파일과 같은 경로에 1.에서 다루었던 titanic_dataset.py와 test.csv, train.csv 파일이 위치해 있는 것을 상정하고 작성한 코드이다.

![1](1-3.PNG)
![2-1](https://imgur.com/MB0vcY3)
![2-2](https://imgur.com/qRX6ELx)

실행 결과이다. 학습이 계속될수록 Training Loss는 줄어듦으로써 학습이 이루어지는 것을 확인할 수 있다. 단, Validation Loss는 들쭉날쭉하여 안정되지 않은 모습이다.

현재의 코드에서는 ReLU를 이용하였다. Torch에서는 ReLU 말고도 ELU, Leaky ReLU, PReLU 등의 다양한 Activation Function을 제공한다. 같은 Epoch를 조건으로, 이들을 이용하여 더 나은 성능을 산출하는 Activation Function이 있는 지 알아보도록 하자. 

![3-1](https://imgur.com/9RE7JGM)
![3-2](https://imgur.com/4cjGHIS)

4개의 Activation Function을 이용하여 학습을 진행한 결과의 Training Loss, Validation Loss를 비교해 본 결과이다. Training Loss부터 언급하자면, Leaky ReLU가 가장 적은 Loss 감소를 보였고, ELU는 초반에는 많은 Loss 감소율을 보였지만 후반으로 갈수록 loss 감소폭이 줄어드는 모습을 보였다. 단, 불안정하거나 급하게 줄어드는 것이 아닌 어느 정도 수렴하여 그 값이 유지되는 모습을 보였다. ReLU와 PReLU는 비슷한 성능을 보였다. 결과적으로는 ReLU와 PReLU가 가장 적은 Training Loss를 보였다.

Validation Loss 면에서는 모든 Activation Function이 어느 정도 불안정한 모습을 보였다. 대체적으로 가장 낮은 loss를 보이는 Activation Function은 ELU였다.

![4-1](https://imgur.com/1PRJeY6)
![4-2](https://imgur.com/8yXQDzf)

Epoch를 10,000으로 늘리고 테스트해본 결과이다. 1,000일때와 반대로 ELU는 지지부진한 Training Loss를 보이는 한편 기존에 제일 부진했던 LeakyReLU가 PReLU와 비슷한 Training Loss 감소율을 보이고 있었다. 
Validation Loss는 ReLU가 제일 적고, 그 뒤를 PReLU가 잇고 있었다. LeakyReLU는 Training Loss는 많이 감소하는 모습이었지만 Validation Loss에서는 그렇지 않았다.

따라서 추후 모델 구성에는 Training Loss가 효과적으로 감소했던 PReLU와 Validation Loss가 제일 적었던 ReLU를 사용해 보기로 했다.

# 3. 테스트 및 submission.csv 생성

2.에서 성능이 제일 좋은 Activation Function을 알아냈으니 이제 이를 통해 모델을 구성할 차례이다.

4가지의 Activation Function을 이용한 훈련 모델을 일정 Epoch만큼 훈련시켜본 결과, 보통 Epoch 후반부가 될 수록 훈련이 어느 정도 진행된다는 것을 Training Loss의 감소로 알 수 있다. 

![5-1](https://imgur.com/WPSnlY0)

그러나 Validation Loss는 대체적으로 그렇지 않다. 이미지는 PReLU를 사용한 학습 결과이다. Training Loss는 시간이 지날수록 계속 감소하여 0에 수렴하는 모습을 보이지만 Validation Loss는 줄어들다가 어느 순간을 기점으로 갑자기 계속 증가하는 것을 확인할 수 있다. 이미지 상에서는 epoch가 2k정도를 전후할때 일어나고 있다. 이렇게 validation loss가 감소하다가 어느 순간부터 계속 증가하는 현상을 과적합(Overfitting) 혹은 과대적합이라고 한다. 과대적합이 발생하는 원인으로는 다음과 같은 예가 있다.

•    훈련 데이터 크기가 너무 작고 가능한 모든 입력 데이터 값을 정확하게 나타내기에 충분한 데이터 샘플을 포함하지 않습니다.
•    훈련 데이터에는 노이즈 데이터라고 하는 관련 없는 정보가 많이 포함되어 있습니다.
•    모델이 단일 샘플 데이터 세트에서 너무 오래 훈련합니다.
•    모델 복잡도가 높기 때문에 훈련 데이터 내의 노이즈를 학습합니다.

(출처: AWS https://aws.amazon.com/ko/what-is/overfitting/)

이런 과적합을 막기 위해서 Early Stopping 방법을 사용하기로 했다. Early Stopping이란 적절한 시점에, 그러니까 과적합이 일어나기 전 시점에서 학습이 잘 수행됐다 판단되면 학습을 조기 종료시키는 것을 말한다. 이를 통해 모델의 과적합을 방지하고 최적의 성능을 얻을 수 있다. 학습이 조기 종료되는 시점으로는 Validation Loss가 향상되지 않거나 오히려 더 나빠지는 시점을 들 수 있다. PyTorch에서 Early Stopping을 구현하려면 Validation Loss를 모니터링하고 Loss가 일정 기간 동안 감소하지 않으면 훈련을 조기 중지하도록 작성한다.

Early Stopping 코드를 추가하여 작성한 코드는 다음과 같다.
Activation Function은 PReLU와 ReLU로 하고, submission.csv는 모든 Epoch가 종료되고 산출할 수 있도록 했다.
또한 Validation Loss가 앞으로 다시 변경될 때를 대비하여 일정 횟수 이상은 loss의 급격한 변화가 있어도 기다려 주는 patience 횟수를 지정했다. 
처음엔 10으로 잡았는데, 학습이 미처 시작되기도 전에 중지되는 문제가 발생하였다. 그래서 500 정도로 넉넉잡아 정하니 학습이 충분히 완료된 시점에서 잘 중지가 되었다.



In [9]:
import torch
from torch import nn, optim
from torch.utils.data import random_split, DataLoader
from datetime import datetime
import wandb
import argparse
from pathlib import Path
import sys
import pandas as pd


# 기본 경로 설정
BASE_PATH = str(Path(__file__).resolve().parent.parent.parent)
sys.path.append(BASE_PATH)
import titanic_dataset as td  # 실제 모듈 이름으로 변경


def get_data():
    # 데이터 전처리 모듈의 함수를 호출하여 데이터 가져오기
    train_dataset, validation_dataset, test_dataset = td.get_preprocessed_dataset()

    train_data_loader = DataLoader(dataset=train_dataset, batch_size=wandb.config.batch_size, shuffle=True)
    validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=wandb.config.batch_size, shuffle=True)
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

    return train_data_loader, validation_data_loader, test_data_loader


# 모델 클래스 정의
class MyModel(nn.Module):
    def __init__(self, n_input, n_output):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]),
            # nn.ReLU(),
            nn.PReLU(),
            nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]), # wandb init 시 지정한 크기대로 계산할 수 있다.
            nn.PReLU(),
            # nn.ReLU(),
            nn.Linear(wandb.config.n_hidden_unit_list[1], n_output),
        )

    def forward(self, x):
        x = self.model(x)
        return x


# EarlyStopping 클래스 정의
class EarlyStopping:
    def __init__(self, patience=500, delta=0, verbose=False, path='checkpoint.pt'):
        self.patience = patience  # patience -> 개선을 위해 몇 번의 Epoch를 대기할 지 지정. 너무 적게 지정하면 훈련이 되기도 전에 종료되는 문제가 있음.
        self.delta = delta # 개선되고 있다고 판단하기 위한 최소 변화량을 나타냄. 만약 변화량이 delta보다 적다면 개선이 없다고 판단.
        self.verbose = verbose # 어느 시점에 훈련을 멈췄는지 화면에 출력
        self.path = path # 모델 저장 경로
        self.counter = 0 # Early Stopping 카운터. 개선이 없었던 경우 1씩 더해 나가 patience만큼 counter가 차면 Early Stopping이 발동된다.
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss, model): # 클래스를 호출했을 때
        if self.best_loss is None:
            self.best_loss = val_loss
            self.save_checkpoint(val_loss, model)  # 최적의 (최소의) loss 값이 없는 경우 val_loss를 체크포인트에 저장
        elif val_loss > self.best_loss + self.delta:  # 최소 변화량과 최적 loss 값을 더한 값보다도 val_loss가 더 크다면 개선이 안 되는 상황으로 판단하고 카운터 += 1.
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:  # counter가 사전 지정한 patience를 넘어간다면 조기 중지.
                if self.verbose:
                    print("Early stopping")
                self.early_stop = True
        else:
            if val_loss < self.best_loss:  # val_loss가 최적 loss값보다 더 적다면 해당 val_loss값을 새로운 최적 loss값으로 지정.
                self.best_loss = val_loss
                self.save_checkpoint(val_loss, model)  # 해당 체크포인트를 저장.
            self.counter = 0

    def save_checkpoint(self, val_loss, model):  # 최적 loss값이 나온 체크포인트를 저장
        if self.verbose:
            print(f'Validation loss decreased ({self.best_loss:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)


# 훈련 루프 함수 수정
def training_loop(model, optimizer, train_data_loader, validation_data_loader):
    n_epochs = wandb.config.epochs
    loss_fn = nn.CrossEntropyLoss()  # 분류 작업을 위한 손실 함수로 수정
    next_print_epoch = 100
    early_stopping = EarlyStopping(verbose=True)

    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        num_trains = 0
        for train_batch in train_data_loader:
            output_train = model(train_batch['input'])
            loss = loss_fn(output_train, train_batch['target'])
            loss_train += loss.item()
            num_trains += 1

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        loss_validation = 0.0
        num_validations = 0
        with torch.no_grad():
            for validation_batch in validation_data_loader:
                output_validation = model(validation_batch['input'])
                loss = loss_fn(output_validation, validation_batch['target'])
                loss_validation += loss.item()
                num_validations += 1

        wandb.log({
            "Epoch": epoch,
            "Training loss": loss_train / num_trains,
            "Validation loss": loss_validation / num_validations
        })

        if epoch >= next_print_epoch:
            print(
                f"Epoch {epoch}, "
                f"Training loss {loss_train / num_trains:.4f}, "
                f"Validation loss {loss_validation / num_validations:.4f}"
            )
            next_print_epoch += 100

        early_stopping(loss_validation, model)

        if early_stopping.early_stop:
            print("Early stopping")
            break

    model.load_state_dict(torch.load('checkpoint.pt'))
    wandb.run.summary["best_validation_loss"] = early_stopping.best_loss




# 모델과 옵티마이저 생성 함수
def get_model_and_optimizer():
    my_model = MyModel(n_input=11, n_output=2)
    optimizer = optim.SGD(my_model.parameters(), lr=wandb.config.learning_rate)

    return my_model, optimizer

def test(test_data_loader, model): # EarlyStopping 발동 이후 테스트
    print("[TEST]")
    prediction = []

    with torch.no_grad():
        for test_batch in test_data_loader:
            output_test = model(test_batch['input'])
            prediction.extend(torch.argmax(output_test,dim=1).tolist())

    return prediction # 테스트 결과 반환

def generate_submission_csv(predictions):
    # 결과 데이터프레임 csv 파일 (submission.csv) 생성
    submission_df = pd.DataFrame({
        "PassengerId": range(892, 892 + len(predictions)),
        "Survived": predictions
    })

    # CSV 파일로 저장
    submission_df.to_csv("submission.csv", index=False)
    print("Submission CSV file generated: submission.csv")


def main(args):
    current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

    config = {
        'epochs': args.epochs,
        'batch_size': args.batch_size,
        'learning_rate': 1e-3,  # 원하는 학습률로 변경
        'n_hidden_unit_list': [30, 30],
    }

    wandb.login()

    wandb.init(
        mode="online" if args.wandb else "disabled",
        project="dl-titanic",
        notes="My first wandb experiment",
        tags=["my_model", "titanic", "kaggle"],
        name=current_time_str,
        config=config
    )

    train_data_loader, validation_data_loader, test_data_loader = get_data()
    linear_model, optimizer = get_model_and_optimizer()

    wandb.watch(linear_model)

    print("[TRAIN]")

    training_loop(
        model=linear_model,
        optimizer=optimizer,
        train_data_loader=train_data_loader,
        validation_data_loader=validation_data_loader,
    )

    # Early Stopping 이후 테스트 수행
    predictions = test(test_data_loader, linear_model)

    # 결과를 submission.csv 파일로 저장
    generate_submission_csv(predictions)

    wandb.finish()


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--wandb", action=argparse.BooleanOptionalAction, default=True, help="True or False"
    )
    parser.add_argument(
        "-b", "--batch_size", type=int, default=16, help="Batch size (int, default: 16)"  # 원하는 배치 크기로 수정
    )
    parser.add_argument(
        "-e", "--epochs", type=int, default=10_000, help="Number of training epochs (int, default: 1,000)"
        # 원하는 에폭으로 수정
    )
    args = parser.parse_args()
    main(args)


NameError: name '__file__' is not defined

Early Stopping 부분과 테스트, csv exporting 과정을 추가한 최종 코드이다.
Early Stopping이 발동되면 즉시 테스트로 넘어가 csv 파일에 대한 테스트를 수행하고, 이 결과를 csv 파일로 내보낸다. 훈련이 끝난 후 모델은 pt 확장자로 저장된다.

In [ ]:
# EarlyStopping 클래스 정의
class EarlyStopping:
    def __init__(self, patience=500, delta=0, verbose=False, path='checkpoint.pt'):
        self.patience = patience  # patience -> 개선을 위해 몇 번의 Epoch를 대기할 지 지정. 너무 적게 지정하면 훈련이 되기도 전에 종료되는 문제가 있음.
        self.delta = delta # 개선되고 있다고 판단하기 위한 최소 변화량을 나타냄. 만약 변화량이 delta보다 적다면 개선이 없다고 판단.
        self.verbose = verbose # 어느 시점에 훈련을 멈췄는지 화면에 출력
        self.path = path # 모델 저장 경로
        self.counter = 0 # Early Stopping 카운터. 개선이 없었던 경우 1씩 더해 나가 patience만큼 counter가 차면 Early Stopping이 발동된다.
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss, model): # 클래스를 호출했을 때
        if self.best_loss is None:
            self.best_loss = val_loss
            self.save_checkpoint(val_loss, model)  # 최적의 (최소의) loss 값이 없는 경우 (초기화 이전인 경우) val_loss를 체크포인트에 저장
        elif val_loss > self.best_loss + self.delta:  # 최소 변화량과 최적 loss 값을 더한 값보다도 val_loss가 더 크다면 개선이 안 되는 상황으로 판단하고 카운터 += 1.
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:  # counter가 사전 지정한 patience를 넘어간다면 조기 중지.
                if self.verbose:
                    print("Early stopping")
                self.early_stop = True
        else:
            if val_loss < self.best_loss:  # val_loss가 최적 loss값보다 더 적다면 해당 val_loss값을 새로운 최적 loss값으로 지정.
                self.best_loss = val_loss
                self.save_checkpoint(val_loss, model)  # 해당 체크포인트를 저장.
            self.counter = 0

    def save_checkpoint(self, val_loss, model):  # 최적 loss값이 나온 체크포인트를 저장
        if self.verbose:
            print(f'Validation loss decreased ({self.best_loss:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)

Early Stopping 기능 구현 부분이다. EarlyStopping 클래스를 호출할 때 val_loss를 계산해 나가도록 하였다. 최초 호출인 경우 best_loss, 즉 최적 loss값의 기준이 되는 값을 초기화하고 현재의 val_loss값, 즉 매 step마다의 loss값을 체크포인트에 저장한다. val_loss가 이전 최적 loss값에 최소 변화량을 더한 값보다도 적다면 카운터에 1을 더한다. 이 카운터가 사전에 지정해놓은 patience, 즉 대기 기준보다 많아질 경우 더 이상 개선되지 않거나 악화되는 것으로 판단하여 훈련은 바로 종료된다. 

val_loss가 현재까지의 최적 loss값보다 낮다면 best_loss를 업데이터하고 새로운 체크포인트를 저장한다.

save_checkpoint는 말 그대로 현재 모델 상태를 지정된 경로에 저장한다. EarlyStopping 클래스를 통해 validation loss값의 개선을 모니터링하며 지정된 조건에 따라 모델 훈련을 조기 중지할 수 있다. 최종 실행 시에는 patience를 500, delta를 0으로 지정하고 학습했다.

In [ ]:
def test(test_data_loader, model): # EarlyStopping 발동 이후 테스트
    print("[TEST]")
    prediction = []

    with torch.no_grad():
        for test_batch in test_data_loader:
            output_test = model(test_batch['input'])
            prediction.extend(torch.argmax(output_test,dim=1).tolist())

    return prediction # 테스트 결과 반환

def generate_submission_csv(predictions):
    # 결과 데이터프레임 csv 파일 (submission.csv) 생성
    submission_df = pd.DataFrame({
        "PassengerId": range(892, 892 + len(predictions)),
        "Survived": predictions
    })

    # CSV 파일로 저장
    submission_df.to_csv("submission.csv", index=False)
    print("Submission CSV file generated: submission.csv")


test는 Early Stopping이 발동된 이후, 즉 훈련이 끝난 직후 실행된다. 과적합이 오기 전 제일 훈련이 잘 되어 체크포인트에 저장된 시점의 모델을 가져와 전처리 후 훈련 데이터, 검증 데이터와 함께 받아왔던 테스트 데이터를 테스트한다. 테스트 후에는 이를 CSV 파일로 변환하여 내보낸다. 변환된 CSV 파일은 Kaggle에 제출하여 점수를 매긴다.

![6-1](https://imgur.com/gRzULOY)

PReLU에 Early Stopping을 적용한 결과

![6-2](https://imgur.com/9Gy1bL0)

ReLU에 Early Stopping을 적용한 결과

PReLU와 ReLU에 Early Stopping을 적용한 결과이다. 둘 다 과대적합이 일어나기 전, 즉 validation loss가 갑자기 증가하는 시점 이전에 학습을 조기 중지하여 train loss가 어느 정도 낮아진 한편 validation loss 역시 낮춰진 상태로 학습을 종료한 것을 확인할 수 있다. Early Stopping을 적용하기 전의 기존 학습보다 확실히 적은 Epoch수에 종료된 것을 확인할 수 있다.

![6-3](https://imgur.com/OKSRVjJ)

학습 이후에는 submission.csv 파일이 생성된다. 이를 Kaggle에 제출하면 된다.


# 4. submission.csv 제출 및 등수 확인

최종 시행 결과로 생성된 submission.csv 파일을 Kaggle에 제출하고 결과를 확인하였다.

![7-1](C:\Users\VRICK\link_dl\link_dl\_03_your_code\_04_learning_and_autograd\7-1.PNG)
![7-2](C:\Users\VRICK\link_dl\link_dl\_03_your_code\_04_learning_and_autograd\7-2.PNG)

Submit 결과이다. 맨 밑은 ReLU 실행 결과, 위의 둘이 PReLU 실행 결과이다. ReLU의 정확도가 더 높게 책정된 것을 확인할 수 있었다.
전체 순위 중에서는 12565위를 기록했다.

# 5. WanDB 페이지 생성 및 URL 제출

해당 과제에서 사용된 WanDB URL은 다음과 같다. URL에서 모든 테스트 결과를 확인할 수 있다.

URL : https://wandb.ai/msb2956/dl-titanic

# 6. 소감

사실 HW2 설명 페이지에 나온 것 이외에도 Torch에서는 많은 Activation Function들을 지원한다. 예를 들면 GELU, SELU, GLU 등이 있다. 이들을 이용한 모델 역시 생성하여 Torch가 지원하는 Activation Function 중 Titanic 예제에는 어떠한 방법이 최적의 방법인지를 알아내고 싶었지만, 학습 시간도 오래 걸리고 여러모로 힘든 작업인지라 해보지 못한 것이 큰 아쉬움으로 남았다.
또한 jupyter에서는 wandb의 arguments를 이용한 실행 방법이 제대로 작동하지 않는 것 같았다. if __name__ == "__main__": 이하의 내용은 명령줄 인수의 입력을 상정한 코드인데, Jupyter는 명령줄 인수로 작동하는 것이 아니다 보니 자꾸 에러가 터지는 것이 관건이었다. 일단 py 파일을 작동시켜도 학습이 진행되고, WanDB에도 기록이 되므로 py 파일을 실행시킨 결과와 WanDB 실행 결과를 첨부하여 실행 여부를 증명하였다.

